# Assessing greenspace accessability within Northern Ireland through an interactive map which calculates the nearest proximity of a settlement to its nearest off-road greensapce trial. 
## Overview 
In this code, you will see how you can use libaries like geopandas and folium in-conjunction with functions like STRtree and nearest_points, to understand how varied greenspace accessability really is within Northern Ireland. Whilst there are more accurate models of greenspace accessability within Northern Ireland, the importance of how accessable a greenspace is goes unnoticed in todays society and although they have great social and cultural values, greenspaces are not accessabile to everyone and therefore, this map shows a robust approach on how you can analyse greensapce accessability. 

## Objectives 
1. Use `geopandas` to __load__ and __re-project__ data files into their correct __EPSG__
2. Use `geodataframe` functions to __store__ settlement data as __centroids__, produce __connection lines__ between the nearest off-road trail and the nearest settlement and finally to __calculate__ the areas of the greenspace polygons
3. Use `shapely` tools including _STRtree_, _nearest_points_ and _LineString_ to calculate the __Euclidean__ distance from each settlement to their closest off-road trail
4. Use `folium` to create an __interactive__ web based map that includes various features like __popups__, __hovering__ over features and a __layer control panel__

## Time to get started

In [163]:
""" 
Imports libaries and tools that are required to conduct the analysis. 
"""
import geopandas as gpd
import folium
from shapely.geometry import LineString
from shapely.strtree import STRtree
from shapely.ops import nearest_points

In [164]:
"""
Loads settlement, trail, and greenspace shapefiles.
"""
settlements = gpd.read_file("data/Settlements/settlements-2015-above-500-threshold.shp")
trails = gpd.read_file("data/Trails/greenspaceoffroadtrails.shp")
greenspace = gpd.read_file("data/Greenspace/greenspace.shp")

In [165]:
"""
Checks the EPSG for the settlements data file.
"""
settlements.crs

<Projected CRS: EPSG:29902>
Name: TM65 / Irish Grid
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: Ireland - onshore.
- bounds: (-10.56, 51.39, -5.93, 55.43)
Coordinate Operation:
- name: Irish Grid
- method: Transverse Mercator
Datum: TM65
- Ellipsoid: Airy Modified 1849
- Prime Meridian: Greenwich

In [166]:
"""
Checks the EPSG for the trails data file.
"""
trails.crs

<Projected CRS: EPSG:29902>
Name: TM65 / Irish Grid
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: Ireland - onshore.
- bounds: (-10.56, 51.39, -5.93, 55.43)
Coordinate Operation:
- name: Irish Grid
- method: Transverse Mercator
Datum: TM65
- Ellipsoid: Airy Modified 1849
- Prime Meridian: Greenwich

In [167]:
"""
Checks the EPSG for the greenspace data file.
"""
greenspace.crs

<Projected CRS: EPSG:29902>
Name: TM65 / Irish Grid
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: Ireland - onshore.
- bounds: (-10.56, 51.39, -5.93, 55.43)
Coordinate Operation:
- name: Irish Grid
- method: Transverse Mercator
Datum: TM65
- Ellipsoid: Airy Modified 1849
- Prime Meridian: Greenwich

In [168]:
"""
Reprojects all spatial layers to EPSG:3857, to ensure that area is calculated in the correct units. 
"""
settlements = settlements.to_crs(epsg=3857)
trails = trails.to_crs(epsg=3857)
greenspace = greenspace.to_crs(epsg=3857)

In [169]:
"""
Re-checks the EPSG for the settlements data file.
"""
settlements.crs

<Projected CRS: EPSG:3857>
Name: WGS 84 / Pseudo-Mercator
Axis Info [cartesian]:
- X[east]: Easting (metre)
- Y[north]: Northing (metre)
Area of Use:
- name: World between 85.06°S and 85.06°N.
- bounds: (-180.0, -85.06, 180.0, 85.06)
Coordinate Operation:
- name: Popular Visualisation Pseudo-Mercator
- method: Popular Visualisation Pseudo Mercator
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [170]:
"""
Re-checks the EPSG for the trails data file.
"""
trails.crs

<Projected CRS: EPSG:3857>
Name: WGS 84 / Pseudo-Mercator
Axis Info [cartesian]:
- X[east]: Easting (metre)
- Y[north]: Northing (metre)
Area of Use:
- name: World between 85.06°S and 85.06°N.
- bounds: (-180.0, -85.06, 180.0, 85.06)
Coordinate Operation:
- name: Popular Visualisation Pseudo-Mercator
- method: Popular Visualisation Pseudo Mercator
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [171]:
"""
Re-checks the EPSG for the greenspace data file.
"""
greenspace.crs

<Projected CRS: EPSG:3857>
Name: WGS 84 / Pseudo-Mercator
Axis Info [cartesian]:
- X[east]: Easting (metre)
- Y[north]: Northing (metre)
Area of Use:
- name: World between 85.06°S and 85.06°N.
- bounds: (-180.0, -85.06, 180.0, 85.06)
Coordinate Operation:
- name: Popular Visualisation Pseudo-Mercator
- method: Popular Visualisation Pseudo Mercator
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [172]:
"""
Displays the first 5 rows of the greenspace shapefile.
"""
greenspace.head(5)

,SourceID,GUID,Name,Source,Category,Type,PaidAccess,Area_Ha,Verified,ShowOnMap,ORNI_ID,DataAdded,SiteCreate,geometry
0,None,{68D2DF20-1512-4218-A759-1000732E93B0},Conservation Volunteers NI,LPS and Outscape,Amenity Greenspace,Community Garden,No,0.668234,Approximated,Yes,51,2022-09-21,Pre 2023,"POLYGON Z ((-658305.131 7272616.366 0, -658467..."
1,None,{7B20F220-682E-4566-A4FE-2513CC65ED6E},Lough Shore Park,Antrim and Newtownabbey Borough Council,Parks and Gardens,Public Park,No,6.598284,Approximated,Yes,61,2022-09-21,Pre 2023,"POLYGON Z ((-694569.114 7306677.129 0, -694647..."
2,None,{8A4B8A31-EBA9-4E7A-B8D5-CEA0724D848D},Ardmore Recreation Centre,"Armagh City, Banbridge and Craigavon Borough C...",Amenity Greenspace,Playing Fields,No,2.969549,Approximated,Yes,67,2022-09-21,Pre 2023,"POLYGON Z ((-737723.904 7234648.427 0, -737711..."
3,None,{36F2A040-649C-4EB5-9174-7B25C083F489},Clare Glen - Phase 3 - Link Footpath/River Wal...,"Armagh City, Banbridge and Craigavon Borough C...",Woodland,Woodland,No,3.396109,Approximated,Yes,69,2022-09-21,Pre 2023,"POLYGON Z ((-713809.418 7236506.822 0, -713812..."
4,None,{2F586E46-B9EE-4D13-83B1-9B254FA3A698},"Folly Glen, Armagh","Armagh City, Banbridge and Craigavon Borough C...",Woodland,Woodland,No,7.849400,Approximated,Yes,70,2022-09-21,Pre 2023,"MULTIPOLYGON Z (((-738837.244 7234074.082 0, -..."


In [173]:
"""
Displays the first 5 rows of the settlements shapefile.
"""
settlements.head(5)

,Code,Name,Band,DT_20MIN,DT_30MIN,UR_ex,OccHH_ex,UR_ap,OccHH_ap,geometry
0,N11000199,DUNNAMORE,H,Y,Y,119.0,35.0,119.0,35.0,"MULTIPOLYGON (((-771836.094 7298554.224, -7717..."
1,N11000015,BALLYBARNES,H,Y,Y,242.0,101.0,243.0,102.0,"POLYGON ((-639797.918 7285752.046, -639794.956..."
2,N11000121,LOUGHGUILE,H,N,Y,396.0,128.0,396.0,128.0,"POLYGON ((-702052.069 7374194.437, -702048.944..."
3,N11000265,BRYANSFORD,H,N,Y,306.0,114.0,309.0,115.0,"POLYGON ((-661378.035 7213541.816, -661374.07 ..."
4,N11000636,SION MILLS,G,Y,Y,1903.0,769.0,1907.0,770.0,"POLYGON ((-832102.671 7319464.147, -832105.467..."


In [174]:
"""
Displays the first 5 rows of the trails shapefile.
"""
trails.head(5)

,ID,Name,SourceID,Source,GUID,Category,Type,Descriptio,On_Road,Length,Verified,ShowOnMap,County,UpdateDate,Trail_Usea,Infrastruc,geometry
0,0,Annalong Coastal Path,166,Outscape,{7a03aac4-145f-436f-9320-45b533fc5930},Single-Use Trail,Walking Trail,Short Walks (up to 5 miles),False,1.055094,None,Yes,Down,None,Walking,False,"LINESTRING Z (-656477.346 7190872.985 0, -6564..."
1,0,Antrim Hills Way,104,Outscape,{c89d5023-1fe8-4def-b9d6-9efdc833540a},Single-Use Trail,Walking Trail,Long Walks (over 20 miles),False,23.081263,None,Yes,Antrim,None,Walking,False,"LINESTRING Z (-662985.466 7355853.417 0, -6629..."
2,0,Ardress House - Ladies Mile,149,Outscape,{0a2fe306-5136-48e1-9a09-067c5dffe2d8},Single-Use Trail,Walking Trail,Short Walks (up to 5 miles),False,0.949078,None,Yes,Armagh,None,Walking,False,"LINESTRING Z (-733785.916 7254732.363 0, -7337..."
3,0,Argory Blackwater River Walk,724,Outscape,{4b801c6c-fab1-4b9e-a475-e2c79be826bc},Single-Use Trail,Walking Trail,Short Walks (up to 5 miles),False,1.546865,None,Yes,Armagh,None,Walking,False,"LINESTRING Z (-740877.981 7258706.199 0, -7408..."
4,0,Argory Lime Tree Walk,319,Outscape,{3c967a01-0486-4f4c-9924-2d9a888e087a},Single-Use Trail,Walking Trail,Short Walks (up to 5 miles),False,0.955559,None,Yes,Armagh,None,Walking,False,"LINESTRING Z (-740877.981 7258706.199 0, -7408..."


In [175]:
"""
Rectifies invalid feature geometries to avoid errors during the distance calculations, spatial indexing when using STRtree and nearest_points and finally, to ensure the map is rendered smoothly.
"""

# Code for validating features is adapted from: https://app.readthedocs.org/projects/shapely/downloads/pdf/latest/
settlements = settlements[settlements.is_valid]
trails = trails[trails.is_valid]
greenspace = greenspace[greenspace.is_valid]

In [176]:
"""
Calculates area, converts it into hectares for each greenspace polygon and stores its data as centroids.
"""
if 'Area_Ha' not in greenspace.columns:
    greenspace['Area_Ha'] = greenspace.geometry.area / 10_000  # Convert m² to ha for an accurate representation of area

greenspace['centroid'] = greenspace.geometry.centroid # Adds a centriod column to the greenspace geo data frame

In [177]:
"""
Uses a spatial index through the STRtree and nearest_points functions to calculate and store the nearest off-road trial distance to the closest settlements. 
"""

# Code implementing STRtree and nearest_points is from: https://shapely.readthedocs.io/en/stable/strtree.html#r77d4c1f42cb7-1
trail_geoms = trails.geometry.tolist() # Converts to a list of individual geometries
trail_tree = STRtree(trail_geoms) # Generate a spatial index 

connection_lines = []  # Creates an empty list to hold LineStrings from settlements to off-road trails
distances = []         # Creates an empty list to hold the calculated Euclidean distances in m

for idx, row in settlements.iterrows(): # Loops through each settlement to process the following details:
    point = row.geometry.centroid # Finds the center point of the settlements centriod 
    nearest_idx = trail_tree.nearest(point) # Find which off-road trail is the closest to each settlement 
    nearest_geom = trail_geoms[nearest_idx] # Finds the geometry of those nearest off-road trials 
    nearest_point = nearest_points(point, nearest_geom)[1] # Finds the point on the off-road trail that is closest to the point on the settlement  

    line = LineString([point, nearest_point]) # Connects the nearest off-road trail point to the nearest settlement point
    distance = point.distance(nearest_point) # Measures the distance from the nearest off-road trails to the nearest settlement

    connection_lines.append(line) # Stores the lines that connects the settlements to the off-road trails
    distances.append(distance) # Saves the distance between the settlements and off-road trails 

connections = gpd.GeoDataFrame(
    {'distance_m': distances}, # Adds the calculated distances as a new attribute called 'distance_m'
    geometry=connection_lines, # Adds the calculates lines to the new attribute
    crs=settlements.crs # Set to use the same CRS as settlements
)

In [178]:
"""
Checks the EPSG of the connections file.
"""
connections.crs

<Projected CRS: EPSG:3857>
Name: WGS 84 / Pseudo-Mercator
Axis Info [cartesian]:
- X[east]: Easting (metre)
- Y[north]: Northing (metre)
Area of Use:
- name: World between 85.06°S and 85.06°N.
- bounds: (-180.0, -85.06, 180.0, 85.06)
Coordinate Operation:
- name: Popular Visualisation Pseudo-Mercator
- method: Popular Visualisation Pseudo Mercator
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [179]:
"""
Reprojects all spatial layers to EPSG:4326 to allow folium to operate.
"""
settlements = settlements.to_crs(epsg=4326)
trails = trails.to_crs(epsg=4326)
greenspace = greenspace.to_crs(epsg=4326)
connections = connections.to_crs(epsg=4326)

In [180]:
"""
Re-checks the EPSG of the settlements data file.
"""
settlements.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [181]:
"""
Re-checks the EPSG of the trails data file.
"""
trails.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [182]:
"""
Re-checks the EPSG of the greenspace data file.
"""
greenspace.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [183]:
"""
Re-checks the EPSG of the connections data file.
"""
connections.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [184]:
"""
Creates a folium web map that centers the map to the centroids of all the settlement points.
"""
center = settlements.geometry.union_all().centroid # Merges the settlements before finding its centroid 
m = folium.Map(location=[center.y, center.x], zoom_start=8) # Sets the web map to open showing the center points of all settlements at a zoom of 8


In [185]:
"""
Adds greenspace polygons to the map with popups revealing its name, category and area.
"""

# Code for lambda styling adapted from: https://medium.com/%40aakash013/geospatial-data-visualization-with-folium-geopandas-092673eeaa35  
for _, row in greenspace.iterrows(): # Loops through each greenspace to process the following details:
    popup = folium.Popup(
        f"<strong>Name:</strong> {row.get('Name', 'N/A')}<br>" # Displays the name of each greenspace 
        f"<strong>Category:</strong> {row.get('Category','N/A')}<br>" # Displays the category of each greenspace 
        f"<strong>Area:</strong> {round(row['Area_Ha'], 2)} ha", # Displays the area of each greenspace 
        max_width=250
    )
    folium.GeoJson(
        row.geometry,
        style_function=lambda x: { # Style used 
            'fillColor': 'darkorange', # Fill colour
            'color': 'darkorange', # Border colour 
            'fillOpacity': 0.4, # Transparency of the polygon 
            'weight': 0.5 # Thickness of border 
        },
        popup=popup # Attaches the specified characteristics to the map
    ).add_to(m) # Adds the style to the map

In [186]:
"""
Adds off-road trail lines to the map that are clickable and show the trail names.
"""
folium.GeoJson(
    trails, # The file that contains the off-road trail entities 
    name='Trails', # The name of the layer in the layer control panel 
    style_function=lambda x: { # The visual style used  
        'color': 'blue', # Off-road trail Colour 
        'weight': 2 # Thickness of off-road trail feature 
    },
    tooltip=folium.GeoJsonTooltip( # Allows the feature to be hovered over 
        fields=['Name'], # Displays the name of the off-road trails            
        aliases=['Trail Name:'], # How the 'Name' is labelled 
        localize=True,
        sticky=True
    )
).add_to(m) # Adds the style to the map

In [187]:
"""
Adds settlements to the map as black circles
"""
for _, row in settlements.iterrows(): # Loops through each settlement to process the following details:
    centroid = row.geometry.centroid # Uses the center of the settlements 
    folium.CircleMarker(
        location=[centroid.y, centroid.x], # Utalises the coordinates of the centroids
        radius=4, # Size of the centroid
        color='black', # Colour of the centroid
        fill=True,
        fill_opacity=0.7, # Transparency of the centroid
        tooltip=row.get("Name", "N/A") # Displays the name of each settlement 
    ).add_to(m) # Adds the style to the map

In [188]:
"""
Draws red lines from each settlement to the nearest off-road trail. Also allows off-road trail lines to be toggled on/off and when hovered over, it will reveal the calculated distances.
"""
folium.GeoJson(
    connections, # The file that contains the connections entities
    name='Nearest Trail Lines', # The name of the layer in the layer control panel 
    style_function=lambda x: { # The visual style used  
        'color': 'red', # Connection lines colour
        'weight': 1.5 # Thickness of the connections feature 
    },
    tooltip=folium.GeoJsonTooltip( # Allows the feature to be hovered over 
        fields=['distance_m'], # Displays the calculated distance of the closest off-road trial to the closest settlement 
        aliases=['Distance (m):'], # How the 'Name' is labelled 
        localize=True
    )
).add_to(m) # Adds the style to the map   

In [189]:
"""
Adds a layer control panel to the web page and exports the map as a HTML file. 
"""

# Code for layer control adapted from: https://python-visualization.github.io/folium/latest/user_guide/ui_elements/layer_control.html
folium.LayerControl().add_to(m) # Enables the layer control panel
m.save("nearest_trail_access_map.html") # Creates the interactive web page 

print('nearest_trail_access_map.html') # Creates a saved file as an output 

nearest_trail_access_map.html


## Conclusion 
I hope that this interactive web based map allows you to see how accessable a greenspace really is to you. If you get stuck, remeber to use the tips in the how-to-guide and I hope that you have taken something away from this code. Thank you!!